In [ ]:
import pymongo
from pymongo import MongoClient
from bson import SON

from IPython.html.widgets import FloatProgress
from IPython.display import display

In [ ]:
client = MongoClient()
db = client['gplaces']
houses_geojson = db['houses_geojson']

In [48]:
import pandas as pd
from pandas import DataFrame

houses_location = pd.read_csv('/root/Downloads/houses_data_loc(1).csv').drop_duplicates(subset='adress')
houses_data = pd.read_csv('/root/hack/Хакатон/Технико-экономические паспорта многоквартирных домов/Данные.csv')

types_houses = 'Год постройки,Общая площадь здания,Число этажей,Количество проживающих,Аварийность дома'.split(',')

for i,loc in houses_location.iterrows():
    newField = {'loc':{ 'type': "Point",'coordinates': [loc['lat'], loc['lng']]}}
    newField['adress'] = loc['adress']
    
    loc_id = loc['id']
    house = houses_data.loc[[loc_id]]
    
    newField['year_of_construction'] = house['Год постройки'].values[0]
    newField['building_area'] = house['Общая площадь здания'].values[0]
    newField['floor_number'] = house['Число этажей'].values[0]
    newField['people_living'] = house['Количество проживающих'].values[0]
    newField['emergency_building'] = house['Аварийность дома'].values[0]
    
    houses_geojson.insert_one(newField)

In [50]:
houses_geojson.find_one()

{'_id': ObjectId('57131c1d6fe8b9424a01f4d6'),
 'adress': 'Санкт-Петербург Лени Голикова ул. д. 23',
 'building_area': 3601.44995117,
 'emergency_building': 0.0,
 'floor_number': 5.0,
 'loc': {'coordinates': [59.844715, 30.229946], 'type': 'Point'},
 'people_living': 213.0,
 'year_of_construction': '1967'}

In [53]:
#DUMP

import json
from bson.json_util import dumps

json_data = list()

for item in houses_geojson.find():
    item_dict = dict()
    item_dict['lng'] = item['loc']['coordinates'][0]
    item_dict['lat'] = item['loc']['coordinates'][1]
    item_dict['count'] = item['people_living'] if item['people_living']>0 else 1
    json_data.extend([item_dict])


with open('data_people_living.json', 'w') as fp:
    json.dump(json_data, fp)

#with open('data_100.json', 'w') as outfile:
#    json.dump(dumps(places_geojson.find().limit(100)), outfile)

In [30]:
houses_geojson.create_index([('loc' , pymongo.GEOSPHERE)])

'loc_2dsphere'

In [49]:

result = houses_geojson.aggregate([
   {
     '$geoNear': {
        'near': { 'type': "Point", 'coordinates': [59.83275469999999, 30.2317329] },
        'distanceField': "dist",
        'maxDistance': 550,
        'spherical': True
     }
   }
])


for i,item in enumerate(result):
    print(item)
    if i > 20:
        break

{'people_living': nan, 'loc': {'type': 'Point', 'coordinates': [59.83737379999999, 30.2303583]}, 'floor_number': 17.0, 'building_area': 15627.799804700002, 'dist': 469.87768996673276, 'emergency_building': 0.0, 'year_of_construction': '2008', 'adress': 'Санкт-Петербург Ветеранов пр. д. 75', '_id': ObjectId('57131c3d6fe8b9424a021ee7')}
{'people_living': 208.0, 'loc': {'type': 'Point', 'coordinates': [59.837202100000006, 30.2299536]}, 'floor_number': 5.0, 'building_area': 3541.51000977, 'dist': 471.3831910879965, 'emergency_building': 0.0, 'year_of_construction': '1967', 'adress': 'Санкт-Петербург Ветеранов пр. д. 67', '_id': ObjectId('57131c206fe8b9424a01f892')}
{'people_living': 171.0, 'loc': {'type': 'Point', 'coordinates': [59.837202100000006, 30.2299536]}, 'floor_number': 9.0, 'building_area': 2393.34008789, 'dist': 471.3831910879965, 'emergency_building': 0.0, 'year_of_construction': '1968', 'adress': 'Санкт-Петербург Ветеранов пр. д. 71', '_id': ObjectId('57131c206fe8b9424a01f896'